## Introdução

O presente notebook aborda o problema de **detecção de fraudes em transações financeiras**. Cada linha da base representa uma transação feita com cartão de crédito, contendo informações como data, valor, localização, estabelecimento, e um campo binário (`is_fraud`) que indica se a transação foi fraudulenta ou não.

Para a tarefa de classificação, foi utilizado o algoritmo **Isolation Forest**, um método não supervisionado especialmente eficaz para detecção de anomalias (outliers).


## Bibliotecas Utilizadas

| Biblioteca / Módulo                        | Descrição                                                                 |
|-------------------------------------------|---------------------------------------------------------------------------|
| `pandas` (`pd`)                           | Utilizada para manipulação e análise de dados em forma de tabelas (DataFrames). |
| `sklearn.model_selection.train_test_split` | Função para dividir o conjunto de dados em treino e teste.                |
| `sklearn.preprocessing.OneHotEncoder`     | Codifica variáveis categóricas em vetores binários (one-hot encoding).    |
| `sklearn.compose.ColumnTransformer`       | Permite aplicar diferentes transformações a diferentes colunas do DataFrame. |
| `sklearn.pipeline.Pipeline`               | Cria um fluxo de processamento com etapas sequenciais (ex: pré-processamento + modelo). |
| `sklearn.ensemble.RandomForestClassifier` | Algoritmo de aprendizado de máquina baseado em múltiplas árvores de decisão para classificação. |
| `sklearn.ensemble.IsolationForest`        | Algoritmo para detecção de anomalias baseado em árvores (unsupervised).   |
| `numpy` (`np`)                            | Biblioteca para computação numérica e manipulação de arrays.              |
| `sklearn.metrics.*`                       | Conjunto de funções para avaliação de modelos (ex: accuracy, F1, ROC AUC, etc.). |
| `matplotlib.pyplot` (`plt`)               | Biblioteca para criação de gráficos e visualizações.                      |


In [ ]:
# Importação de bibliotecas
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import IsolationForest
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.metrics import (
    classification_report,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    average_precision_score
)
import matplotlib.pyplot as plt

## Descrição das Colunas da Base de Dados

| Coluna                 | Descrição                                                                 |
|------------------------|---------------------------------------------------------------------------|
| `Unnamed: 0`           | Índice original do dataset                                                |
| `trans_date_trans_time` | Data e hora da transação.                                                |
| `cc_num`               | Número do cartão de crédito (anonimizado).                                |
| `merchant`             | Nome do comerciante onde a transação ocorreu.                             |
| `category`             | Categoria do comerciante (ex: food, gas_transport, etc).                  |
| `amt`                  | Valor da transação em dólares.                                            |
| `first`                | Primeiro nome do titular do cartão.                                       |
| `last`                 | Sobrenome do titular do cartão.                                           |
| `gender`               | Gênero do titular do cartão (`M` ou `F`).                                 |
| `street`               | Endereço residencial do titular do cartão.                                |
| `lat`                  | Latitude da residência do titular do cartão.                              |
| `long`                 | Longitude da residência do titular do cartão.                             |
| `city_pop`             | População estimada da cidade de residência.                               |
| `job`                  | Profissão do titular do cartão.                                           |
| `dob`                  | Data de nascimento do titular do cartão.                                  |
| `trans_num`            | Identificador único da transação.                                         |
| `unix_time`            | Timestamp UNIX da transação.                                              |
| `merch_lat`            | Latitude do comerciante.                                                  |
| `merch_long`           | Longitude do comerciante.                                                 |
| `is_fraud`             | Indicador de fraude (`1` para fraude, `0` para legítima).                 |


In [ ]:
# Importação e visualização das colunas
dataframe = pd.read_csv('/kaggle/input/fraud-detection/fraudTrain.csv')
dataframe.head()

## Organização de dados relevantes

1. Transformar datas em  conjuntos de objetos em formato datetime.
2. Organização de transações por cartões, em seguida, calcula a diferença de tempo entre cada transação.
3. Remove colunas irrelevantes.
4. Categorizando colunas em classes `Numéricas` e `Categóricas`.

### Separação da Variável `X`

| Variável | Conteúdo                              | Função no Modelo                                |
|----------|----------------------------------------|--------------------------------------------------|
| `X`      | Todas as colunas independentes do DataFrame (features) que serão usadas para encontrar anomalias. A variável alvo (target_col, como is_fraud) não é incluída para o treinamento. | Usada como entrada para o treinamento do Isolation Forest, que aprende a identificar desvios nos dados. |

In [ ]:
# Datas para Objetos Datetimes
dataframe['trans_date_trans_time'] = pd.to_datetime(dataframe['trans_date_trans_time'])
dataframe['hour_of_day'] = dataframe['trans_date_trans_time'].dt.hour
dataframe['day_of_week'] = dataframe['trans_date_trans_time'].dt.dayofweek
dataframe['month'] = dataframe['trans_date_trans_time'].dt.month

In [ ]:
# Diferença de tempo entre transações.
dataframe = dataframe.sort_values(by=['cc_num', 'trans_date_trans_time'])
dataframe['time_since_last_transaction'] = dataframe.groupby('cc_num')['trans_date_trans_time'].diff().dt.total_seconds().fillna(0)

In [ ]:
# Definir coluna label (Será usada para testar os resultados gerado pelo Modelo)
target_col = 'is_fraud'

# Remover colunas irrelevantes e label dos dados
columns_to_drop_final = ['Unnamed: 0', 'trans_num', 'first', 'last', 'street', 'city', 'zip', 'dob', 'ssn']
df_processed = dataframe.drop(columns=columns_to_drop_final, errors='ignore').copy()

# Separar X e y
X = df_processed.drop(columns=[target_col], errors='ignore')
y = df_processed[target_col]

# Definir colunas numéricas e categóricas
numerical_cols = ['amt', 'lat', 'long', 'city_pop', 'unix_time',
                  'time_since_last_transaction', 'merch_lat', 'merch_long']
categorical_cols = ['category', 'gender', 'state', 'job', 'merchant']

# Dividir conjunto de dados
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

## Treinamento

1. Pré-Processamento de dados para o modelo usar de forma eficiente as classes `numéricas` e `categóricas`
2. Separação de treino e testes. Neste caso, foi utilizado 80% dos dados para testes e 20% para validações do modelo.
3. Treinamento do modelo utilizando o modelo Isolation Forest.

In [ ]:
# Calculo de contaminação
fraud_ratio = y_train.sum() / len(y_train)
contamination_rate = max(fraud_ratio, 0.01)

# Preprocessamento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='drop'
)

# Treinamento do Modelo
iso_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('anomaly_detector', IsolationForest(
        n_estimators=100,
        contamination=contamination_rate,
        random_state=42,
        n_jobs=-1
    ))
])

iso_pipeline.fit(X_train)

# Classificando de acordo com o modelo
X_test_preprocessed = iso_pipeline.named_steps['preprocessor'].transform(X_test)
anomaly_scores = -iso_pipeline.named_steps['anomaly_detector'].decision_function(X_test_preprocessed)

# Ajuste de limite (threshold)
threshold = np.percentile(anomaly_scores, 99.5)
iso_predictions = (anomaly_scores >= threshold).astype(int)
iso_predictions = 1 - iso_predictions


## Avaliação do Modelo

In [ ]:

# Avaliação do Modelo
print("\nRelatório de Classificação (Isolation Forest):\n",
      classification_report(y_test, iso_predictions, target_names=['Não Fraude', 'Fraude']))

# Métricas
accuracy_iso = accuracy_score(y_test, iso_predictions)
precision_iso = precision_score(y_test, iso_predictions, pos_label=1)
recall_iso = recall_score(y_test, iso_predictions, pos_label=1)
f1_iso = f1_score(y_test, iso_predictions, pos_label=1)

iso_anomaly_scores = -iso_pipeline.named_steps['anomaly_detector'].decision_function(
    iso_pipeline.named_steps['preprocessor'].transform(X_test)
)

scaled_scores = MinMaxScaler().fit_transform(iso_anomaly_scores.reshape(-1, 1)).flatten()

roc_auc_iso = roc_auc_score(y_test, scaled_scores)
pr_auc_iso = average_precision_score(y_test, scaled_scores)

# Impressão dos resultados
print(f"Acurácia: {accuracy_iso:.4f}")
print(f"Precisão (Fraude): {precision_iso:.4f}")
print(f"Recall (Fraude): {recall_iso:.4f}")
print(f"F1-Score (Fraude): {f1_iso:.4f}")
print(f"AUC-ROC: {roc_auc_iso:.4f}")
print(f"AUC-PR: {pr_auc_iso:.4f}")

## Conclusão do Modelo Inicial Isolation Forest

# Análise dos Resultados:

O modelo de Isolation Forest apresenta uma acurácia extremamente baixa (1.04%). Entretanto:

1. A base é altamente desbalanceada: menos de 1% são fraudes.

2. A precisão (0.56%) para fraude é péssimo, pois o modelo praticamente não acerta, logo, quando ele diz que é fraude, geralmente está errado.

3. O Recall (97.07%) para fraude é excelente: Este é o único ponto positivo notável do modelo. Um recall de 97.07% indica que o modelo consegue identificar quase todas as fraudes reais. Ele é muito bom em "pegar" as fraudes que realmente existem. No entanto, este alto recall vem à custa de uma precisão extremamente baixa, como vimos acima.

4. F1-Score (0.011%) para fraude é muito baixo.
